In [4]:
from pyspark.sql import SparkSession
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row,DataFrameWriter
from pyspark.sql.functions import lit, col, regexp_replace
import os
import glob

In [15]:
#   SEÇÃO DO PYSPARK
spark = SparkSession.builder\
    .master('local[*]')\
    .appName('animescsv')\
    .getOrCreate()

In [5]:
# FUNÇÃO DE EXPORTAÇÃO DOS REGISTROS PARA BASE DE DADOS SQL
def exportar_para_oracle(dataframe,nome_tabela,password):
    dataframe.write.format("jdbc") \
    .option("url", "jdbc:oracle:thin:@//localhost:1521/xe") \
    .option("dbtable", nome_tabela) \
    .option("user", "system") \
    .option("password",password ) \
    .option("driver", "oracle.jdbc.driver.OracleDriver") \
    .mode("overwrite") \
    .save()

In [27]:
pasta =os.getcwd()
arquivos_csv = glob.glob(pathname=os.path.join(pasta, '*.csv'))

#for arquivo in arquivos_csv:
#    print(arquivo)
df = spark.read.option('anime.csv', ",").\
                    option("header", True).\
                    csv('anime.csv')
                    
if 'MAL_ID' in df.columns:
    df = df.drop('MAL_ID')
                    


In [21]:
df.show(10)

+------+--------------------+-----+--------------------+--------------------+------------------------------+-----+--------+--------------------+-----------+--------------------+--------------------+----------------+--------+---------------+--------------------+------+----------+-------+---------+--------+---------+-------+-------+-------------+--------+--------+--------+-------+-------+-------+-------+-------+-------+-------+
|MAL_ID|                Name|Score|              Genres|        English name|                 Japanese name| Type|Episodes|               Aired|  Premiered|           Producers|           Licensors|         Studios|  Source|       Duration|              Rating|Ranked|Popularity|Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10| Score-9| Score-8|Score-7|Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+------+--------------------+-----+--------------------+--------------------+------------------------------+-----+--------+-----------------

In [9]:
# CONTROLE DE SENHA, COLUNAS NÃO DESEJADAS E EXPORTAÇÃO
senha_sql   = str(input('INFORME A SENHA DO BANCO DE DADOS: '))

df = df.drop("Japanese name","Producers","Score-10", "Score-9","Score-8","Score-7",
                                        "Score-6","Score-5","Score-4","Score-3","Score-2","Score-1")

exportar_para_oracle(dataframe=df, nome_tabela='ANIMES_LISTA',password=senha_sql)

+------+--------------------+-----+--------------------+--------------------+-----+--------+--------------------+-----------+--------------------+----------------+-----------+---------------+--------------------+------+----------+-------+---------+--------+---------+-------+-------+-------------+
|MAL_ID|                Name|Score|              Genres|        English name| Type|Episodes|               Aired|  Premiered|           Licensors|         Studios|     Source|       Duration|              Rating|Ranked|Popularity|Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|
+------+--------------------+-----+--------------------+--------------------+-----+--------+--------------------+-----------+--------------------+----------------+-----------+---------------+--------------------+------+----------+-------+---------+--------+---------+-------+-------+-------------+
|     1|        Cowboy Bebop| 8.78|Action, Adventure...|        Cowboy Bebop|   TV|      26|Apr 3, 1998 to